# Ensemble Methods: Bagging and Boosting

## Introduction

*  For bagging, we will continue to use the same customer churn dataset that we used to build decision tree classifiers and regressors in the last practical. We will first train a random forest classifier and find a good number of base estimators by means of cross validation using: (i) oob evaluation and (ii) grid search. The performance of the classifier on customer churn prediction is then evaluated using the test set. This is followed by training of an Extra-trees classifier and comparison of the performance of the random forest and Extra-trees models. <br>
*  For boosting, we will switch to a breast cancer dataset to train an AdaBoost classifier and a gradient boosting classifier.  We will use grid search to find optimal combinations of key hyperparameters (no. of base learners and learning rate) for both classifiers. We will also experiment with the early stopping technique in gradient boosting. We will examine and compare the performance of these classifiers.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 1. Random forests and Extra-trees on customer churn dataset

*  We will train a random forest classifier on the telecom customer churn dataset <br>
*  The objective is to predict customer churn (true or false) based on the customers' service usage attributes in the dataset <br>
(Some parts of the code in this section are adopted from Reference [1])

### 1.1 Data exploration and preparation

We are already familiar with the dataset from our last practical. As random forests and Extra-trees are decision tree-based models, the data preparation requirements are the same as for decision trees. Here, we will quickly go through the same steps as we did in the last practical to prepare the dataset. <br>
* Import the customer churn data <br>
* Remove the columns that are unlikely to be useful for prediction <br>
* Encode nominal data ('yes'/'no') to boolean values <br>

In [ ]:
data = pd.read_csv('Orange_Telecom_Churn_Data.csv')

Remove the attribues `state`, `phone_number` and `area_code` as they are unlikely to be useful predictors

In [ ]:
data.drop(['state', 'phone_number', 'area_code'], axis=1, inplace=True)

Encode `intl_plan` and `voice_mail_plan` as True/False as tree-based models in Scikit-Learn are unable to handle categorical data

In [ ]:
for col in ['intl_plan', 'voice_mail_plan']:
    data[col] = data[col].replace('yes',True).replace('no',False).astype(np.bool)
      
data[['intl_plan', 'voice_mail_plan']].dtypes

As the dataset is skewed (about 86% are non-churned customers), we will apply stratified split to divide the dataset into the training and test sets

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Separate feature columns and the target column
feature_cols = [x for x in data.columns if x != 'churned']

# Split the data into two parts with 1500 points in the test data

# This creates a generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=1500, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['churned']))

# Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'churned']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'churned']

In [ ]:
y_train.value_counts()

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts()

In [ ]:
y_test.value_counts(normalize=True)

Data preparation is done <br>
Now, build random forests and Extra-trees

In [ ]:
# Suppress warnings about too few trees from the early models

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def measure_error(y_true, y_pred, label):
    return pd.Series({'accuracy':accuracy_score(y_true, y_pred),
                      'precision': precision_score(y_true, y_pred),
                      'recall': recall_score(y_true, y_pred),
                      'f1': f1_score(y_true, y_pred)},
                      name=label)

### 1.2 Random forests

* Fit random forest models with a range of tree numbers and evaluate the out-of-bag error for each of these models <br>
* Plot the resulting oob errors as a function of the number of trees <br>
* Since the only thing changing is the number of trees, the `warm_start` flag can be used so that the model just adds more trees to the existing model each time <br>
* Use the `set_params` method to update the number of trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the random forest estimator
# Note that the number of trees is not setup here

rf_clf = RandomForestClassifier(oob_score=True, 
                                random_state=42, 
                                warm_start=True,
                                n_jobs=-1)

oob_list = list()

# Iterate through a range of numbers of trees

for n_trees in [10, 50, 100, 500, 600, 800, 1000, 1100, 1200, 1300, 1400, 1500]:
    
    # Use this to set the number of trees
    rf_clf.set_params(n_estimators=n_trees)

    # Fit the model
    rf_clf.fit(X_train, y_train)

    # Get the oob error
    oob_error = 1 - rf_clf.oob_score_
    
    # Store it
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob = pd.concat(oob_list, axis=1).T.set_index('n_trees')

rf_oob

The error appears to have stabilized from around 1100 trees <br>
Plot the graph for easy visualization

In [ ]:
sns.set_context('talk')
sns.set_palette('dark')
sns.set_style('white')

ax = rf_oob.plot(legend=False, marker='o')
ax.set(ylabel='out-of-bag error');

### 1.3 Extra-trees

* Repeat Section 1.2 using Extremely Randomized Trees (`ExtraTreesClassifier`) <br>
* Note that the `bootstrap` parameter will have to be set to `True` for this model, to indicate that we want to use bagging <br>
* Compare the out-of-bag errors for the two models

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# Initialize the Extra-trees estimator
# Note that the number of trees is not setup here

et_clf = ExtraTreesClassifier(oob_score=True, 
                              random_state=42, 
                              warm_start=True,
                              bootstrap=True,
                              n_jobs=-1)

oob_list = list()

# Iterate through a range of numbers of trees
for n_trees in [10, 50, 100, 500, 600, 800, 1000, 1100, 1200, 1300, 1400, 1500]:
       
    # Use this to set the number of trees
    et_clf.set_params(n_estimators=n_trees)
    et_clf.fit(X_train, y_train)

    # oob error
    oob_error = 1 - et_clf.oob_score_
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

et_oob = pd.concat(oob_list, axis=1).T.set_index('n_trees')

et_oob

Combine the two dataframes into a single one for easier plotting

In [ ]:
oob_df = pd.concat([rf_oob.rename(columns={'oob':'RandomForest'}),
                    et_oob.rename(columns={'oob':'ExtraTrees'})], axis=1)

oob_df

Plot the graph

In [ ]:
sns.set_context('talk')
sns.set_palette('dark')
sns.set_style('white')

ax = oob_df.plot(marker='o')
ax.set(ylabel='out-of-bag error');

The random forest model performs consistently better than the Extra-trees

### 1.4 Evaluation of random forest classifier

We choose the random forest model <br>
First, instantiate a random forest with 1100 base estimators that gives the lowest oob error

In [ ]:
# Random forest with 1100 estimators

rf_clf = rf_clf.set_params(n_estimators=1100, warm_start=False)

As in decision trees, we can plot the feature importances <br>
They are not exactly, but more or less, the same as those obtained from the decision trees in the last practical

In [ ]:
feature_imp = pd.Series(rf_clf.feature_importances_, index=feature_cols).sort_values(ascending=False)
fig = plt.figure(figsize=(12,5))
ax = feature_imp.plot(kind='bar')
ax.set(ylabel='Relative Importance');

Let's evaluate the performance of the random forest model using cross-validation

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(rf_clf, X_train, y_train, cv=3, method="predict")
y_train_pred_prob = cross_val_predict(rf_clf, X_train, y_train, cv=3, method="predict_proba")

Compute the error metrics on cross-validation results

In [ ]:
train_error = pd.concat([measure_error(y_train, y_train_pred, 'train')],
                         axis=1)

train_error

Compute the confusion matrix

In [ ]:
cm = confusion_matrix(y_train, y_train_pred)

sns.set_context('talk')
ax = sns.heatmap(cm, annot=True, fmt='d')
labels = ['False', 'True']
ax.set_xticklabels(labels);
ax.set_yticklabels(labels);
ax.set_ylabel('Actual');
ax.set_xlabel('Predicted');

In [ ]:
print(classification_report(y_train, y_train_pred))

What would be the performance of this classifier on the test set?

In [ ]:
y_test_pred = rf_clf.predict(X_test)

In [ ]:
test_error = pd.concat([measure_error(y_test, y_test_pred, 'test')],
                       axis=1)

test_error

The results are comparable with the test results of optimized decision tree models built in the last practical, but recall for positive class (churned customers) is still not great

As in the last practical, we can take a look at the predicted probability of training instances versus their true labels

In [ ]:
y_train_pred

In [ ]:
y_train_pred_prob

In [ ]:
y_train = y_train.replace(True, 1).replace(False, 0).astype(np.int64)

y_train_scores = y_train_pred_prob[:, 1]

train_predict = pd.DataFrame({'actual': y_train.values,
                              'predict': y_train_scores})

train_predict.plot.scatter(x='predict', y='actual', s=list(range(2,500)), alpha=0.05, figsize=(10,6));

The distribution looks better than what we could get from the decision tree models in the last practical <br>
It seems like there is a chance to achieve a better balance of precision and recall for both classes by adjusting the decision threshold in terms of the prediction probability as we did with the decision trees in the last practical

Let's plot the precision/recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train, y_train_scores)

In [ ]:
def plot_precision_recall_curve(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper right", fontsize=16)
    plt.xlim([0, 1])
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_curve(precisions, recalls, thresholds)
plt.show()

Try decision threshold at 0.3

In [ ]:
y_train_pred = (y_train_scores > 0.3)

In [ ]:
train_error = pd.concat([measure_error(y_train, y_train_pred, 'train')],
                         axis=1)

train_error

In [ ]:
print(confusion_matrix(y_train, y_train_pred))

Looks promising <br>
Let's predict on the test set

In [ ]:
y_test_pred_prob = rf_clf.predict_proba(X_test)

In [ ]:
y_test_pred_prob

In [ ]:
y_test_scores = y_test_pred_prob[:, 1]

Set the same threshold for predicting the test instances

In [ ]:
y_test_pred = (y_test_scores > 0.3)

In [ ]:
test_error = pd.concat([measure_error(y_test, y_test_pred, 'test')],
                        axis=1)

test_error

In [ ]:
print(confusion_matrix(y_test, y_test_pred))

This set of balanced results is a huge improvement from what we could get from the decision tree models of the last practical, even without compensating for skewed class distribution

### 1.5 Optimized random forest classifier based on grid search

Alternatively, instead of using oob evaluation, we can also determine the optimal number of trees using grid search (a technique that we've learned in the last practical)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500]}

gr_rf_clf = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1),
                         param_grid=param_grid,
                         scoring='accuracy',
                         n_jobs=-1)

gr_rf_clf = gr_rf_clf.fit(X_train, y_train)

In [ ]:
gr_rf_clf.best_params_

The number of base estimators is not far from what we got from the oob evaluation (Section 1.4)

In [ ]:
y_train_pred = gr_rf_clf.predict(X_train)
y_test_pred = gr_rf_clf.predict(X_test)
# y_test_pred_prob = gr_rf_clf.predict_proba(X_test)

train_test_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                              measure_error(y_test, y_test_pred, 'test')],
                              axis=1)

train_test_error

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)

sns.set_context('talk')
ax = sns.heatmap(cm, annot=True, fmt='d')
labels = ['False', 'True']
ax.set_xticklabels(labels);
ax.set_yticklabels(labels);
ax.set_ylabel('Actual');
ax.set_xlabel('Predicted');

In [ ]:
print(classification_report(y_test, y_test_pred))

The performance is similar to the model obtained through oob evaluation (before adjusting the deceision threshold) in Section 1.4, which is expected

We have experimented with the `class_weight` method to deal with the imbalanced dataset when constructing the decision tree model <br>
You may try a similar technique in random forests by setting the hyperparameter `class_weight='balanced_subsample'` when creating the `RandomForestClassifier()` instance <br>
Interest learners may refer to this article: https://machinelearningmastery.com/bagging-and-random-forest-for-imbalanced-classification/

## 2. Boosting with AdaBoost and gradient boosting
* In this part of the practical, we will be using the Wisconsin Diagnostic breast cancer dataset, which is one of the datasets that we can load from Scikit-Learn <br>
* The dataset consists of 569 training samples and 30 features <br>
* The features are tumor attributes captured from medical images, and the target attribute to be predicted is whether the tumor is malignant or benign <br>
* We will learn to build an AdaBoost classifier and a gradient boosting classifier with this dataset

### 2.1 Data exploration and preparation

As usual, we need to understand more about the dataset and take the necessary steps to prepare the data for our machine learning task

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

First, load the dataset

In [ ]:
from sklearn.datasets import load_breast_cancer

bc = load_breast_cancer() 

Most datasets available from Scikit-Learn have a `DESCR` property that gives a detailed description about the dataset

In [ ]:
bc.DESCR

What are the other properties available?

In [ ]:
bc.keys()

What are the features in this dataset?

In [ ]:
bc.feature_names

In [ ]:
len(bc.feature_names)

What are the target labels?

In [ ]:
bc.target_names

The data is stored with the `data` key

In [ ]:
bc.data.shape

The target column is stored with the `target` key

In [ ]:
bc.target.shape

We want to combine the data and target into a data frame, and name the target column as 'malignant'

In [ ]:
data = np.c_[bc.data, bc.target]
column_names = np.append(bc.feature_names, ['malignant'])
data = pd.DataFrame(data, columns=column_names)

In [ ]:
data.shape

In [ ]:
data.info()

Take note that the target labels are already numerically coded

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data['malignant'].unique()

In [ ]:
data['malignant'].value_counts()
data['malignant'].value_counts(normalize = True)

From the `DESCR` (Class Distribution: 212 - Malignant, 357 - Benign), we know that 'Malignant' is currently coded as 0.0, and 'Benign' as 1.0

We want to recode 'Malignant' to True (positive class) and 'Benign' to False

In [ ]:
data['malignant'] = (data['malignant'] == 0.0).astype(bool)

In [ ]:
data.info()

In [ ]:
data['malignant'].value_counts()
data['malignant'].value_counts(normalize=True)

As both AdaBoost and gradient boosting are tree-based models, data scaling is not necessary

However, as the dataset is slightly skewed, we should use the stratified train/test split method

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

feature_cols = [x for x in data.columns if x != 'malignant']

# Create the generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['malignant']))

# Create the training and test sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'malignant']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'malignant']

In [ ]:
y_train.value_counts()
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts()
y_test.value_counts(normalize=True)

Data preparation is complete <br>
Now build the models

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def measure_error(y_true, y_pred, label):
    return pd.Series({'accuracy': accuracy_score(y_true, y_pred),
                      'precision': precision_score(y_true, y_pred),
                      'recall': recall_score(y_true, y_pred),
                      'f1': f1_score(y_true, y_pred)},
                      name=label)

### 2.2 AdaBoost classifier

Create an AdaBoost classifier and fit it using grid search <br> 
Try a range of number of estimators and learning rates <br>

NOTE: Setting `max_features=4` in the decision tree base estimators will increase the convergence rate

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1, max_features=4, random_state=42),
                             random_state=42)

param_grid = {'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400],
              'learning_rate': [0.3, 0.1, 0.01]}

gr_ada_clf = GridSearchCV(ada_clf,
                          param_grid=param_grid, 
                          scoring='accuracy',
                          n_jobs=-1)

gr_ada_clf = gr_ada_clf.fit(X_train, y_train)

The best model found by the grid search

In [ ]:
gr_ada_clf.best_estimator_

In [ ]:
gr_ada_clf.best_params_

In [ ]:
y_train_pred = gr_ada_clf.predict(X_train)
y_test_pred = gr_ada_clf.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                   measure_error(y_test, y_test_pred, 'test')],
                                   axis=1)
print(train_test_full_error)
print()

print(classification_report(y_test, y_test_pred))
print()

cm = confusion_matrix(y_test, y_test_pred)

sns.set_context('talk')
ax = sns.heatmap(cm, annot=True, fmt='d')
labels = ['False', 'True']
ax.set_xticklabels(labels);
ax.set_yticklabels(labels);
ax.set_ylabel('Actual');
ax.set_xlabel('Predicted');

### 2.3 Gradient boosting classifier

Creare a gradient boosting classifier and fit it with grid search <br>
In grid search, we try to vary the two key hyperparameters, `n_estimators` and `learning_rate`

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 200, 300, 400], 'learning_rate': [0.1, 0.01]}

gr_gb_clf = GridSearchCV(GradientBoostingClassifier(max_features=4, random_state=42),
                      param_grid=param_grid,
                      scoring='accuracy',
                      n_jobs=-1)

gr_gb_clf = gr_gb_clf.fit(X_train, y_train)

In [ ]:
gr_gb_clf.best_estimator_

In [ ]:
gr_gb_clf.best_params_

In [ ]:
y_train_pred = gr_gb_clf.predict(X_train)
y_test_pred = gr_gb_clf.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                   measure_error(y_test, y_test_pred, 'test')],
                                   axis=1)
print(train_test_full_error)
print()

print(classification_report(y_test, y_test_pred))
print()

cm = confusion_matrix(y_test, y_test_pred)

sns.set_context('talk')
ax = sns.heatmap(cm, annot=True, fmt='d')
labels = ['False', 'True']
ax.set_xticklabels(labels);
ax.set_yticklabels(labels);
ax.set_ylabel('Actual');
ax.set_xlabel('Predicted');

Another technique for searching optimal results is early stopping <br>
We set the hyperparameter `n_iter_no_change` to enable early stopping

In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=400, learning_rate=0.01, max_features=4, 
                                    n_iter_no_change=50, random_state=42)

In [ ]:
gb_clf.fit(X_train, y_train)

The actual number of base learners trained (the higher number of base learners is due to the lower learning rate set)

In [ ]:
gb_clf.n_estimators_

In [ ]:
y_train_pred = gb_clf.predict(X_train)
y_test_pred = gb_clf.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                   measure_error(y_test, y_test_pred, 'test')],
                                   axis=1)
print(train_test_full_error)
print()

print(classification_report(y_test, y_test_pred))
print()

cm = confusion_matrix(y_test, y_test_pred)

sns.set_context('talk')
ax = sns.heatmap(cm, annot=True, fmt='d')
labels = ['False', 'True']
ax.set_xticklabels(labels);
ax.set_yticklabels(labels);
ax.set_ylabel('Actual');
ax.set_xlabel('Predicted');

Although the accuracy of higher than 97% is nice, we would hope that the false negative cases (now 3) could be lower. What could we possibly investigate further? <br>

#### References
[1] Intel AI Academy, Machine Learning 501. <br>